<a href="https://colab.research.google.com/github/pmsorion/etlv4/blob/main/etlFlotas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import psycopg2
from psycopg2 import pool
from psycopg2.extras import RealDictCursor

In [3]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np

In [73]:
try:
  postgresQL_pool = psycopg2.pool.SimpleConnectionPool(1, 20, user="metrics", 
                                                       password="BOsb8j0Au05ehlPw",
                                                       host="la-mamalona.99minutos.app",
                                                       port="5432",
                                                       database="pr99minutoscom")
  if (postgresQL_pool):
    print("Connection pool created successfully")

  # use getcon() to get cnnection from connection pool
  ps_connection = postgresQL_pool.getconn()

  if (ps_connection):
    print("sucdessfully recived connection from connection pool")
    ps_cursor = ps_connection.cursor(cursor_factory=RealDictCursor)
    dateString = '2021-01-01'


    sqlSentence = """SELECT CAST(o.id AS INTEGER) AS orderid,
                                        COALESCE(CAST(o.tripid AS VARCHAR), '0') AS idtrip,
                                        CAST(o.trackingid AS VARCHAR) AS trackingid,
                                        CAST(o.counter AS BIGINT) AS counter,
                                        COALESCE(CAST(o.receivedorderid AS VARCHAR), '0') AS receivedorderid,
                                        COALESCE(CAST(o.packages AS INTEGER), 0) AS packages,
                                        CAST(o.packagesize AS VARCHAR) AS packagesize,
                                        COALESCE(CAST(o.finalprice AS DECIMAL), 0.0) AS finalprice,
                                        COALESCE(CAST(o.finalgrossprice AS DECIMAL), 0.0) AS finalgrossprice,
                                        COALESCE(CAST(o.grossprice AS DECIMAL), 0.0) AS grossprice,
                                        COALESCE(CAST(o.price AS DECIMAL), 0.0) AS price,
                                        COALESCE(CAST(o.insured AS DECIMAL), 0.0) AS insured,
                                        COALESCE(CAST(o.amount AS DECIMAL), 0.0) AS costamount,
                                        COALESCE(CAST(o.cost AS DECIMAL), 0.0) AS costpay,
                                        CAST(o.isrecolect AS BOOL) AS recolect,
                                        CAST(o.returnc AS BOOL) AS returnc,
                                        CAST(o.isreturnable AS BOOL ) AS isreturnable,
                                        CAST(o.haspaid AS BOOL) AS haspaid,
                                        COALESCE(CAST(regexp_replace(o.barcode, E'[\\n\\r]+', ' ', 'g' ) AS VARCHAR), 'None') AS barcode,
                                        CAST(replace(o.notes, ',', '') AS VARCHAR) AS notesorder,
                                        CAST(COALESCE(o.driverid, 0) AS INTEGER) AS driverid,
                                        CAST(o.billingid AS INTEGER) AS billingid,
                                        CAST(o.rating AS INTEGER) AS rating,
                                        CAST(o.created AS TIMESTAMP) AS created,
                                        CAST(o.currentstatus AS VARCHAR) AS currentstatus,
                                        COALESCE(CAST(o.zone_id AS VARCHAR), '') AS zone_id,
                                        COALESCE(CAST(o.parentorder AS BIGINT), 0) AS parentorder,
                                        CAST(o.created_at AS TIMESTAMP) AS created_at_orders,
                                        COALESCE(CAST(lp.id AS INTEGER), 0) AS logid,
                                        CAST(lp.statusid AS VARCHAR) AS status,
                                        COALESCE(CAST(lp.driverid AS INTEGER), 0) AS driverstatus,
                                        CAST(lp.comment AS VARCHAR) AS comment,
                                        CAST(lp.note AS VARCHAR) AS notas,
                                        COALESCE(CAST(lp.commentclient AS VARCHAR), 'None') AS commentclient,
                                        CAST(lp.created AS TIMESTAMP) AS datestatus,
                                        CAST(lp.created_at AS TIMESTAMP) AS created_at_logsorders,
                                        CAST(replace(cda.route, ',', '') AS VARCHAR) AS destinationroute,
                                        CAST(cda.zipcode AS VARCHAR) AS destinationzipcode,
                                        CASE
                                            WHEN char_length(cda.lat) = 0 THEN CAST(0 AS FLOAT)
                                            ELSE CAST(COALESCE(substring(cda.lat, 1, 10), '0') AS FLOAT)
                                        END AS destinationlat,
                                        CASE
                                            WHEN char_length(cda.long) = 0 THEN CAST(0 AS FLOAT)
                                            ELSE CAST(COALESCE(substring(cda.long, 1, 10), '0') AS FLOAT)
                                        END AS destinationlong,
                                        COALESCE(CAST(cda.neighborhood AS VARCHAR), 'None') AS coloniadestination,
                                        CAST(replace(cpa.route, ',', '') AS VARCHAR) AS pickuproute,
                                        CAST(cpa.zipcode AS VARCHAR) AS pickupzipcode,
                                        CASE
                                            WHEN char_length(cpa.lat) = 0 THEN CAST(0 AS FLOAT)
                                            ELSE CAST(COALESCE(substring(cpa.lat, 1, 10), '0') AS FLOAT)
                                        END AS pickuplat,
                                        CASE
                                            WHEN char_length(cpa.long) = 0 THEN CAST(0 AS FLOAT)
                                            ELSE CAST(COALESCE(substring(cpa.long, 1, 10), '0') AS FLOAT)
                                        END AS pickuplong,
                                        COALESCE(CAST(cpa.neighborhood AS VARCHAR), 'None') AS coloniaorigin,
                                        CAST(ctd.id AS INTEGER) AS typeid,
                                        COALESCE(CAST(d.userid AS INTEGER), 0) AS iduserdriver,
                                        COALESCE(CAST(d.phone AS VARCHAR), 'None') AS driverphone,
                                        COALESCE(CAST(d.driversuser AS VARCHAR), 'None') AS driversuser,
                                        COALESCE(CAST(d.driversname AS VARCHAR), 'None') AS driversname,
                                        COALESCE(CAST(d.assigns AS VARCHAR), 'None') AS assigns,
                                        COALESCE(CAST(d.vehiclesid AS INTEGER), 0) AS vehiclesid,
                                        COALESCE(CAST(d.basesalary AS DECIMAL), 0) AS basesalary,
                                        COALESCE(CAST(ctv.vehicle AS VARCHAR), 'None') AS vehicle,
                                        CAST(u.id AS INTEGER) AS userid,
                                        CAST(u.company AS VARCHAR) AS company,
                                        CAST(u.country AS VARCHAR) AS country,
                                        CAST(u.role AS VARCHAR) AS role,
                                        CAST(u.created AS TIMESTAMP) AS usercreated,
                                        CAST(u.lastlogin AS TIMESTAMP) AS userlastlogin,
                                        CAST(u.parentid AS INTEGER) AS parentid,
                                        COALESCE(CAST(u.size_id AS INTEGER), 0) AS size_id,
                                        COALESCE(CAST(csta.id AS INTEGER), 0) AS stationid,
                                        COALESCE(CAST(csta.code AS VARCHAR), 'None') AS stationcode,
                                        TO_CHAR(o.created :: time, 'HH24:MI:SS') AS createdtime,
                                        CAST(o.datedelivery AS TIMESTAMP) AS datedelivery,
                                        CAST(CASE
                                            WHEN  CAST(TO_CHAR(o.created :: time, 'HH24:MI:SS') AS TIME ) >= '15:00:00' AND
                                                CAST(TO_CHAR(o.created :: time, 'HH24:MI:SS') AS TIME ) <= '23:00:00'
                                            THEN 'En Rango'
                                            ELSE 'Fuera'
                                            END AS VARCHAR)
                                        AS deliverysuccess,
                                        CAST(DATE_PART('day', o.created :: timestamp - o.datedelivery :: timestamp) * 24 +
                                        DATE_PART('hour', o.created :: timestamp - o.datedelivery :: timestamp) AS INTEGER)
                                        AS datediff                                      
                                FROM  orders o LEFT JOIN logsorders lp ON o.id = lp.orderid
                                        LEFT JOIN catdestinationadress cda ON cda.id = o.destionationadressid
                                        LEFT JOIN catpickupadress cpa ON cpa.id = o.pickupadressid
                                        LEFT JOIN catypedeliveries ctd ON ctd.id = o.deliverytypeid
                                        LEFT JOIN drivers d ON d.id = o.driverid
                                        LEFT JOIN catvehicles ctv ON ctv.id = d.vehiclesid
                                        LEFT JOIN useraccounts u ON o.userid = u.id
                                        LEFT JOIN catstation csta ON csta.id = lp.station_id
                                WHERE   o.created BETWEEN '{0} 00:00:00' and '{0} 23:59:59'
                                ORDER BY o.id asc""".format(dateString)
    ps_cursor.execute(sqlSentence)
    data_records = ps_cursor.fetchall()
    data_frame = pd.DataFrame(data_records)

except (Exception, psycopg2.DatabaseError) as error:
  print("error while connection to PostgreSQL", error)

finally:
  # closing database connection
  # use closeall() method to close all the active connection If you wnat to turn of the application
  if postgresQL_pool:
    postgresQL_pool.closeall
  print("postgreSQL connection pool is closed")

Connection pool created successfully
sucdessfully recived connection from connection pool
postgreSQL connection pool is closed


In [74]:
print(list(data_frame))

['orderid', 'idtrip', 'trackingid', 'counter', 'receivedorderid', 'packages', 'packagesize', 'finalprice', 'finalgrossprice', 'grossprice', 'price', 'insured', 'costamount', 'costpay', 'recolect', 'returnc', 'isreturnable', 'haspaid', 'barcode', 'notesorder', 'driverid', 'billingid', 'rating', 'created', 'currentstatus', 'zone_id', 'parentorder', 'created_at_orders', 'logid', 'status', 'driverstatus', 'comment', 'notas', 'commentclient', 'datestatus', 'created_at_logsorders', 'destinationroute', 'destinationzipcode', 'destinationlat', 'destinationlong', 'coloniadestination', 'pickuproute', 'pickupzipcode', 'pickuplat', 'pickuplong', 'coloniaorigin', 'typeid', 'iduserdriver', 'driverphone', 'driversuser', 'driversname', 'assigns', 'vehiclesid', 'basesalary', 'vehicle', 'userid', 'company', 'country', 'role', 'usercreated', 'userlastlogin', 'parentid', 'size_id', 'stationid', 'stationcode', 'createdtime', 'datedelivery', 'deliverysuccess', 'datediff']


In [75]:
data_frame[['status' , 'notas', 'createdtime']]

,status,notas,createdtime
0,6,Terminado (devolución) por: erick.gomez en la ...,00:49:03
1,4,Cambiado a la estación por Pamela el número de...,00:49:03
2,2,Recolectada por el veloz Don Veloz CHL 2020101...,00:49:03
3,100,Orden 84981031001 Recoleccion catalina.mellad...,00:49:03
4,1,"#id{""OrderNumber"": ""402534548809"", ""CustomerOr...",00:49:03
...,...,...,...
10132,1,#id | Cobro: No | Recibe: Juan Eduardo Romo L...,23:57:51
10133,4,Cambiado a la estación por operarios.tultitlan...,23:59:10
10134,6,Finalizada por el veloz DON VELOZ IS TULTI 000,23:59:10
10135,1,#id | Cobro: No | Recibe: Christopher Hafid Ga...,23:59:10


In [76]:
data_frame.dtypes

orderid                     int64
idtrip                     object
trackingid                 object
counter                     int64
receivedorderid            object
                        ...      
stationcode                object
createdtime                object
datedelivery       datetime64[ns]
deliverysuccess            object
datediff                    int64
Length: 69, dtype: object

In [54]:
import datetime as dt

In [ ]:
data_frame['notas'].str.findall('Orden')

0             []
1             []
2             []
3             []
4        [Orden]
          ...   
88120         []
88121         []
88122         []
88123         []
88124         []
Name: notas, Length: 88125, dtype: object

In [ ]:
#data_frame['notas'] = data_frame['notas'].str.findall('Orden')


In [ ]:
#data_frame['notas'] = data_frame['notas'].str.replace(data_frame['notas'].str.findall('Orden'), 'Orden del dia')
data_frame['notas'].str.contains('Orden')

0        False
1        False
2        False
3        False
4         True
         ...  
88120    False
88121    False
88122    False
88123    False
88124    False
Name: notas, Length: 88125, dtype: bool

In [ ]:
motivo = {
    'Cliente Ausente', 
    'Cliente No Localizable', 
    'Domicilio Incorrecto', 
    'Rechaza Paquete', 
    'Robo', 
    'Extravio', 
    'Establecimiento Cerrado', 
    'El Mensajero No Llega Por Tráfico',
    'Daño De Paquete',
    'Cliente Recoge Paquete En Sucursal'
    }

In [ ]:
for mot in motivo:
  data_frame.loc[data_frame['notas'].str.contains(mot), 'notas'] = mot

In [ ]:
data_frame.loc[data_frame.status.isin(['3', '10', '17']), 'notas'] = 'Auto Reasignado'

In [ ]:
data_frame.loc[data_frame['notas'].str.contains('Orden'), 'notas'] = 'Orden del dia'

In [ ]:
data_frame[['status' , 'notas']]

,status,notas
0,1,#id10000034428 | Cobro: No | Recibe: nancy la...
1,1,#id7353829 | Cobro: No | Recibe: Nicolas Vene...
2,1,#id2000149710 | Cobro: No | Recibe: CAROLINA ...
3,6,"carlos, Titular,"
4,3,Auto Reasignado
...,...,...
88120,1,#id1123041238181832 | Cobro: No | Recibe: null...
88121,1,#id1123041238181865 | Cobro: No | Recibe: null...
88122,1,#id1123041238181873 | Cobro: No | Recibe: null...
88123,1,#id1123041238181881 | Cobro: No | Recibe: null...
